In [253]:
import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


from xgboost import XGBClassifier, XGBRegressor

In [266]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

test_df = test_df.drop(['Income'],axis=1)

val_instances = test_df['Instance']


Y = train_df['Income in EUR'].values
X_raw = train_df.drop(['Income in EUR'], axis=1)


# This is kinda ugly but w/e... Load in X_test here too, and combine into one massive X dataset.
# Then can later separate for training. Helps labels be generated correctly and all that.

X_raw = pd.concat([X_raw, test_df],axis='rows')

# Drop instance labels.
X_raw = X_raw.drop(['Instance'], axis=1)


# Fill in null years & ages
X_raw['Year of Record'] = X_raw['Year of Record'].fillna(2000)
X_raw['Age'] = X_raw['Age'].fillna(20)

print(X_raw)

       Year of Record   Gender   Age                   Country  Size of City  \
0              1997.0        0  41.0                   Belarus       1239930   
1              1996.0    other  41.0                 Singapore       1603504   
2              2018.0    other  28.0                    Norway       1298017   
3              2006.0    other  33.0                      Cuba        751903   
4              2010.0   female  46.0      United Arab Emirates         95389   
5              1991.0     male  60.0                   Liberia         56080   
6              1982.0      NaN  71.0        State of Palestine       1205185   
7              2008.0     male  36.0                    Israel         31419   
8              2015.0     male  43.0               South Sudan         85333   
9              2019.0   female  16.0                      Cuba       1325383   
10             1981.0     male  51.0                Kyrgyzstan       2362346   
11             1989.0    other  15.0    

In [267]:
# Country GDP Data
from iso3166 import countries
import pycountry


# Create map from countries in dataset to country code.
df_countries = X_raw['Country']
keys = df_countries.unique()
print(keys)

countrycode = {}
for k in keys:
    try:
        c = pycountry.countries.lookup(k)
        countrycode[k] = c.alpha_3
        
    except LookupError:
        #print("couldn't find", k)
        # All just syria cause w/e
        countrycode[k] = 'SYR'
        
        
df_gdp = pd.read_csv('GDP_Data.csv')
df_gdp = df_gdp.loc[df_gdp['MEASURE'] == 'MLN_USD']

cGood = 0
cBad = 0

year_ave = {}
for y in range(1980, 2019):
    ave = df_gdp.loc[df_gdp['TIME'] == y]['Value'].mean()
    year_ave[y] = ave
    #print(y, ave)
    
year_ave[2019] = year_ave[2018]

def lookup(row):
    global cGood
    global cBad
    
    if cGood % 100 == 0:
        print(cGood, cBad)
    
    #print(row)
    
    year = row['Year of Record']
    code = countrycode[row['Country']]
    
    try:
        #v = df_gdp[('LOCATION' == code) & ('TIME' == year) & ('MEASURE' == 'MLN_USD')]
        v = df_gdp[(df_gdp['LOCATION'] == code) & (df_gdp['TIME'] == year)].iloc[0]['Value']

        #print(v)
        cGood += 1
        return v
    except IndexError:
        #print("Couldn't find..", code, year)
        cBad = cBad + 1
        # If no info, just give year average...
        # Try returning a fraction of it, since the countries not included in the list
        # are more likely to be poorer.
        return year_ave[2000]



gdp = X_raw.apply(lambda row: lookup(row), axis=1)
print(gdp)

['Belarus' 'Singapore' 'Norway' 'Cuba' 'United Arab Emirates' 'Liberia'
 'State of Palestine' 'Israel' 'South Sudan' 'Kyrgyzstan' 'Togo' 'Finland'
 'Sierra Leone' 'Papua New Guinea' 'Czechia' 'Paraguay' 'Belgium'
 'Costa Rica' 'Senegal' 'Congo' 'Slovakia' 'Burundi' 'Portugal' 'Tunisia'
 'Lebanon' 'Azerbaijan' 'Chile' 'Botswana' 'Jordan' 'Zimbabwe' 'Rwanda'
 'Greece' 'Turkmenistan' 'Sweden' 'El Salvador' 'Somalia' 'Haiti'
 'Tajikistan' 'Hungary' 'Peru' 'Laos' 'Austria' 'Bolivia' 'Kazakhstan'
 'Nicaragua' 'Moldova' 'Gabon' 'Bulgaria' 'Namibia' 'Croatia'
 'Central African Republic' 'Ghana' 'Denmark' 'Cambodia' 'Netherlands'
 'Niger' 'Mozambique' 'Ireland' 'Mongolia' 'Luxembourg' 'Benin' 'Kuwait'
 'Serbia' 'Chad' 'North Korea' 'Dominican Republic' 'Burkina Faso'
 'Switzerland' 'Ecuador' 'Honduras' 'Libya' 'Syria' 'Uruguay' 'Qatar'
 'Mauritania' 'Jamaica' 'Guinea-Bissau' 'Panama' 'Cameroon'
 'Bosnia and Herzegovina' 'North Macedonia' 'Oman' 'Eritrea' 'Australia'
 'Guinea' 'Saudi Arabia' 'Al

13200 37372
13300 37704
13400 38030
13500 38318
13500 38319
13500 38320
13500 38321
13600 38623
13600 38624
13600 38625
13600 38626
13600 38627
13600 38628
13600 38629
13600 38630
13700 38960
13800 39236
13800 39237
13800 39238
13800 39239
13900 39442
13900 39443
14000 39729
14000 39730
14100 40030
14200 40296
14200 40297
14300 40601
14300 40602
14300 40603
14300 40604
14300 40605
14400 40833
14500 41166
14600 41390
14700 41702
14800 41958
14900 42217
15000 42465
15000 42466
15100 42743
15200 42978
15200 42979
15300 43225
15300 43226
15300 43227
15300 43228
15300 43229
15300 43230
15300 43231
15400 43454
15400 43455
15400 43456
15500 43721
15500 43722
15500 43723
15600 44010
15700 44345
15700 44346
15700 44347
15800 44633
15800 44634
15800 44635
15800 44636
15800 44637
15800 44638
15800 44639
15800 44640
15900 44971
15900 44972
16000 45258
16000 45259
16100 45503
16100 45504
16100 45505
16100 45506
16200 45788
16300 46064
16300 46065
16400 46326
16400 46327
16400 46328
16500 46602
1660

31700 89744
31700 89745
31700 89746
31800 90007
31800 90008
31900 90323
31900 90324
31900 90325
31900 90326
31900 90327
32000 90646
32000 90647
32100 90884
32100 90885
32100 90886
32100 90887
32100 90888
32100 90889
32100 90890
32100 90891
32100 90892
32200 91153
32200 91154
32200 91155
32200 91156
32200 91157
32200 91158
32300 91471
32300 91472
32300 91473
32400 91738
32400 91739
32500 92003
32600 92231
32600 92232
32600 92233
32600 92234
32600 92235
32700 92583
32700 92584
32800 92865
32800 92866
32900 93139
32900 93140
32900 93141
32900 93142
32900 93143
32900 93144
33000 93488
33000 93489
33000 93490
33000 93491
33000 93492
33000 93493
33000 93494
33000 93495
33100 93801
33100 93802
33100 93803
33100 93804
33100 93805
33100 93806
33100 93807
33200 94063
33200 94064
33200 94065
33200 94066
33200 94067
33300 94359
33300 94360
33300 94361
33300 94362
33300 94363
33300 94364
33300 94365
33300 94366
33300 94367
33300 94368
33300 94369
33300 94370
33300 94371
33300 94372
33300 94373
3330

47800 135572
47800 135573
47800 135574
47800 135575
47800 135576
47800 135577
47800 135578
47800 135579
47800 135580
47900 135871
47900 135872
47900 135873
47900 135874
47900 135875
47900 135876
47900 135877
47900 135878
47900 135879
48000 136153
48000 136154
48000 136155
48000 136156
48000 136157
48000 136158
48000 136159
48000 136160
48000 136161
48000 136162
48000 136163
48000 136164
48100 136486
48100 136487
48100 136488
48100 136489
48100 136490
48100 136491
48100 136492
48100 136493
48100 136494
48100 136495
48200 136768
48200 136769
48200 136770
48200 136771
48200 136772
48200 136773
0        1.828322e+06
1        1.828322e+06
2        3.481453e+05
3        1.828322e+06
4        1.828322e+06
5        1.828322e+06
6        1.828322e+06
7        2.005028e+05
8        1.828322e+06
9        1.828322e+06
10       1.828322e+06
11       1.828322e+06
12       1.119387e+05
13       1.828322e+06
14       1.828322e+06
15       1.828322e+06
16       1.828322e+06
17       3.222456e+05
18    

In [268]:
    
df_gpi = pd.read_csv("API_NY.GNP.PCAP.CD_DS2_en_csv_v2_248110.csv")
c = 0
def lookupGPI(row):
    global c
    if c % 1000 == 0:
        print(c)
    c += 1
    
    year = row['Year of Record']
    if year == np.nan:
        year = 2000
    if year > 2018:
        year = 2018
    code = countrycode[row['Country']]
    
    #try:
    v = df_gpi[(df_gpi['Country Code'] == code)].iloc[0][str(int(year))]
    #print(year, code, v)
    if v == np.nan:
        v = 200
    return v
    
    
gpi = X_raw.apply(lambda row: lookupGPI(row), axis=1)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [270]:
# Preprocess... Assemble our X - input to training.
X = pd.DataFrame()

year = X_raw['Year of Record']
# Normalize
X['Year'] = (year-year.min())/(year.max() - year.min())
X['YearsAgo'] = 2019 - year

# Gender... 0 = female, 1 = other/NaN/0, 2 = male
# TODO, replace with catagorical?
gender = X_raw['Gender']
gender = gender.replace([0, '0', np.nan], "unknown")
dummy_gender = (pd.get_dummies(gender))
X = pd.concat([X, dummy_gender], axis=1)

# Country
country_dum = pd.get_dummies(X_raw['Country']) #159 countries
X = pd.concat([X, country_dum], axis=1)

# Age
X['Age'] = X_raw['Age']

# Professions
prof = X_raw['Profession'].str.lower()
profession_dum = pd.get_dummies(prof) # 1339

# Drop rare professions
vc = prof.value_counts()
rare_prof = vc[vc <= 5].index.tolist()
print(rare_prof)
profession_dum = profession_dum.drop(rare_prof, axis=1)
X = pd.concat([X, profession_dum], axis=1)


# Adding parameter for 'is assistant', engineer, scientist etc.
X['isassistant'] = 0
X['isassistant'].loc[prof.str.contains("assistant") == True] = 1

X['isengineer'] = 0
X['isengineer'].loc[prof.str.contains("engineer") == True] = 1


# Degrees
deg = X_raw['University Degree']
deg = deg.replace([0, '0'], 'No')
degree_dum = pd.get_dummies(deg) # 4 degrees
X = pd.concat([X, degree_dum], axis=1)

# HairCol
haircol_dum = pd.get_dummies(X_raw['Hair Color'])
X = pd.concat([X, haircol_dum], axis=1)

X['CityPop'] = X_raw['Size of City']
X['CityPoplog'] = np.log(X_raw['Size of City'])

X['Height'] = X_raw['Body Height [cm]']
X['Heightsq'] = np.power(X['Height'], 2)

X['YesGlasses'] = X_raw['Wears Glasses']
X['NoGlasses'] = 1-X_raw['Wears Glasses']

# Just bulletproof by adding adding in default vals.
X['GDP'] = gdp.fillna(4000)
X['GPI'] = gpi.fillna(200)

# Just fill in all other nans with 0 cause .. o well.
X.fillna(0)

print(X)

['bridge operator', 'business manager', 'claim specialist', 'bookbinder', 'brewery manager', 'collector', 'accountable project manager', 'assembler', 'chief executive', 'communication worker', 'chief engineer of dispute resolutions', 'accountable manager', 'computer and information systems manager', 'account manager', 'agricultural scientist', 'community & industry relations associate', 'agricultural engineer', 'appraiser', 'agency counsel', 'cabinetmaker', 'accounting investment analyst', 'brickmason', 'commercial diver', 'bookkeeper', 'avionics technician', 'compliance auditor', 'compliance analyst', 'administrative coordinator', 'civilian investigator', 'building control', 'city planner', 'business solution architect', 'climber & pruner', 'animal control worker', '.net software developer', 'civil design lead', 'chief operating officer', 'boiler inspector', 'admin engineer', 'cafeteria attendant', 'chief plan examiner', 'chief information security officer', 'atmospheric scientist', '

In [25]:

df_gdp = pd.read_csv('GDP_Data.csv')
print(df_gdp.keys)

<bound method NDFrame.keys of      LOCATION INDICATOR SUBJECT  MEASURE FREQUENCY  TIME          Value  \
0         AUS       GDP     TOT  MLN_USD         A  1960   25036.166800   
1         AUS       GDP     TOT  MLN_USD         A  1961   25326.382200   
2         AUS       GDP     TOT  MLN_USD         A  1962   27913.205700   
3         AUS       GDP     TOT  MLN_USD         A  1963   30385.723200   
4         AUS       GDP     TOT  MLN_USD         A  1964   32695.878300   
5         AUS       GDP     TOT  MLN_USD         A  1965   34489.925000   
6         AUS       GDP     TOT  MLN_USD         A  1966   38285.310600   
7         AUS       GDP     TOT  MLN_USD         A  1967   41004.015900   
8         AUS       GDP     TOT  MLN_USD         A  1968   46488.435200   
9         AUS       GDP     TOT  MLN_USD         A  1969   52131.937100   
10        AUS       GDP     TOT  MLN_USD         A  1970   58750.172000   
11        AUS       GDP     TOT  MLN_USD         A  1971   64018.24770

In [278]:
# xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
#                     silent=True, nthread=1)

xgb = XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,       # Was: 10000                                                              
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

xgb = XGBRegressor(colsample_bytree=0.4,
                   eval_metric='rmse',
                 gamma=0,
                   nthread=16,
                   verbosity=1,
                 learning_rate=0.1,
                 max_depth=5,             # Was: 3
                 min_child_weight=1,
                 n_estimators=1000,       # Was: 10000                                                              
                 #reg_alpha=0.75,
                 #reg_lambda=0.45,
                 #subsample=0.6,
                 seed=42)

# Best so far...
xgb = XGBRegressor(colsample_bytree=0.4,
                   eval_metric='rmse',
                 gamma=0,
                   nthread=8,
                   verbosity=2,
                 learning_rate=0.1,
                 max_depth=5, 
                 min_child_weight=1,
                 n_estimators=1300,                                                     
                 #reg_alpha=0.75,
                 #reg_lambda=0.45,
                 #subsample=0.6,
                 seed=42)

# xgb = XGBRegressor(colsample_bytree=0.4,  # 77810 to beat
#                    eval_metric='rmse',
#                  gamma=0,
#                    nthread=8,
#                    verbosity=2,
#                  learning_rate=0.1,
#                  max_depth=5,
#                  min_child_weight=1,
#                  n_estimators=1000,       # Was: 10000                                                              
#                  reg_alpha=0.75,
#                  reg_lambda=0.45,
#                  subsample=0.6,
#                  seed=42)

In [281]:



# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }
params = {
        'min_child_weight': [5],
        'gamma': [1],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'max_depth': [5]
        }

#random_search = GridSearchCV(estimator=xgb, param_grid=params, scoring='neg_mean_squared_error', n_jobs=6, verbose=10)

# Here we go
#start_time = timer(None) # timing starts from this point for "start_time" variable
print("Startin'!")
#random_search.fit(X, Y)

# Split out our validation stuff from X...

X_partial = X.iloc[0:111993,:] 
X_val     = X.iloc[111993:,:]

X_train, X_test, y_train, y_test = train_test_split(X_partial, Y, test_size=0.01, random_state=42)


# Clip y to not be so large, reducing outliers.
#y_train = Y #Y.clip(0,1000000)


print("he", xgb.fit(X_partial, Y))
print("Done")

Startin'!
[00:38:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:38:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[00:38:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[00:39:00] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:39:00] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[00:39:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:39:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[00:39:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 e

[00:39:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[00:39:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:39:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[00:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[00:39:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:39:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[00:39:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nod

[00:40:21] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:40:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[00:40:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[00:40:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[00:40:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[00:40:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[00:40:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:40:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nod

[00:40:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[00:41:00] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[00:41:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[00:41:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[00:41:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:41:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[00:41:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:41:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nod

[00:41:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[00:41:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[00:41:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[00:41:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[00:41:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[00:41:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:41:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[00:41:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nod

...

[00:50:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[00:50:26] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[00:50:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[00:50:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:50:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[00:50:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[00:50:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[00:50:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nod

[00:51:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:51:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:51:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[00:51:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[00:51:21] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[00:51:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[00:51:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:51:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nod

[00:51:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:52:00] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:52:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[00:52:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:52:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[00:52:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:52:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:52:04] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nod

[00:52:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:52:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[00:52:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[00:52:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[00:52:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[00:52:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[00:52:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[00:52:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nod

[00:53:18] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[00:53:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[00:53:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[00:53:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[00:53:21] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[00:53:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[00:53:22] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[00:53:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nod

In [282]:
y_train_pred = xgb.predict(X_train)
RMSE = np.sqrt(mean_squared_error(y_train, y_train_pred))

y_test_pred = xgb.predict(X_test)
RMSE2 = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(RMSE.round(4), RMSE2.round(4))

52234.3565 57808.2223


In [166]:
skf = KFold(n_splits=folds, shuffle = True, random_state = 1001)

a = model_selection.cross_val_score(xgb, X, Y, cv=skf, scoring='neg_mean_squared_error')
print([np.sqrt(-v) for v in a])

[21:34:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[21:34:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[21:34:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[21:34:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[21:34:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[21:34:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[21:34:26] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes

KeyboardInterrupt: 

In [252]:
#print(X['GPI'])


TypeError: '(slice(None, None, None), Year                              False
YearsAgo                          False
female                            False
male                              False
other                             False
unknown                           False
Afghanistan                       False
Albania                           False
Algeria                           False
Angola                            False
Argentina                         False
Armenia                           False
Australia                         False
Austria                           False
Azerbaijan                        False
Bahamas                           False
Bahrain                           False
Barbados                          False
Belarus                           False
Belgium                           False
Belize                            False
Benin                             False
Bhutan                            False
Bolivia                           False
Bo

In [283]:
# Make predictions & SAVE!!!!!!!!!!!
y_val = xgb.predict(X_val)

submission = pd.DataFrame({
        "Instance": val_instances,
        "Income": y_val.flatten()
    })
submission = pd.DataFrame.from_records(submission)
submission.to_csv('submission.csv', index=False)
